In [1]:
from src.utils.data_utils import load_dataframe_from_csv
from src.data.dataloader import load_initial_dataset
from clean_cmu import clean_movies_cmu
#from config import *

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
movie_cmu = load_dataframe_from_csv('movie_cmu.csv')
character = load_dataframe_from_csv('character.csv')
plot_summaries = load_dataframe_from_csv('plot_summaries.csv')
tvtropes = load_dataframe_from_csv('tvtropes.csv')
name_clusters = load_dataframe_from_csv('name_clusters.csv')
oscar_winning_films = load_dataframe_from_csv('oscar_winning_films_ids.csv')
oscar_winning_actors = load_dataframe_from_csv('oscar_winning_actors.csv')
oscar_winning_actresses = load_dataframe_from_csv('oscar_winning_actresses.csv')
oscar_supporting_actors = load_dataframe_from_csv('oscar_winning_supporting_actors.csv')
oscar_supporting_actresses = load_dataframe_from_csv('oscar_winning_supporting_actresses.csv')
extended_films = load_dataframe_from_csv('film_2015_2024.csv')
academy_award_winning_films = load_dataframe_from_csv('acedemy_award_winning_films')
winning_actors_info = load_dataframe_from_csv('winning_actors_information.csv')

In [3]:
extended_films["Movie genres"] = extended_films["genres"]
extended_films["Movie release date"] = extended_films["release_date"]
extended_films["Movie languages"] = extended_films["languages"]
extended_films["Movie countries"] = extended_films["countries"]
extended_films["Movie name"] = extended_films["film"]
extended_films["Movie box office revenue"] = extended_films["box_office"]
extended_films["Movie runtime"] = extended_films["runtime"]
extended_films["Wikipedia movie ID"] = extended_films["page_id"]
extended_films = extended_films.drop(columns=["genres", "release_date", "languages", "countries", "release date", "film", "box_office", "runtime", "page_id"])
winning_actors_info["Actor name"] = winning_actors_info["actor"]
winning_actors_info = winning_actors_info.drop(columns=["actor"])

In [4]:
#oscar_actors_movies = movie[movie['Wikipedia movie ID'].isin(oscar_winning_actors['film_id'])]
#
#oscar_actors_movies = pd.merge(oscar_actors_movies, oscar_winning_actors, left_on='Wikipedia movie ID', right_on='film_id')
#oscar_actors_movies = oscar_actors_movies.drop(columns=["film_id"])
#
#oscar_actors_extended = extended_films[extended_films['Wikipedia movie ID'].isin(oscar_winning_actors['film_id'])]
#oscar_actors_extended = pd.merge(oscar_actors_extended, oscar_winning_actors, left_on='Wikipedia movie ID', right_on='film_id')
#oscar_actors_extended = oscar_actors_extended.drop(columns=["film_id"])
#
oscar_supp_actors_movies_cmu  = pd.merge(oscar_supporting_actors, movie_cmu, left_on='film_id', right_on='Wikipedia movie ID')
oscar_supp_actors_movies_cmu.drop(columns=['film_id'], inplace=True) 

oscar_supp_actors_movies_cmu.shape

(76, 10)

In [5]:
print(oscar_supp_actors_movies_cmu.shape)
print(oscar_supp_actors_movies_cmu['Movie release date'].min())
print(oscar_supp_actors_movies_cmu['Movie release date'].max())

(76, 10)
1939.0
2012.0


In [6]:
oscar_supp_actors_movies_extended  = pd.merge(oscar_supporting_actors, extended_films, left_on='film_id', right_on='Wikipedia movie ID')
oscar_supp_actors_movies_extended.drop(columns=['film_id'], inplace=True) 

# Convert the 'Movie release date' column to datetime
oscar_supp_actors_movies_extended['Movie release date'] = pd.to_datetime(oscar_supp_actors_movies_extended['Movie release date'], format='%Y-%m-%dT%H:%M:%SZ')

# Extract the year
oscar_supp_actors_movies_extended['Movie release date'] = oscar_supp_actors_movies_extended['Movie release date'].dt.year

In [7]:
print(oscar_supp_actors_movies_extended.shape)
print(oscar_supp_actors_movies_extended['Movie release date'].min())
print(oscar_supp_actors_movies_extended['Movie release date'].max())

(7, 14)
2015
2023


In [8]:
oscar_supp_actors_movies = pd.concat([oscar_supp_actors_movies_cmu, oscar_supp_actors_movies_extended], axis=0)
oscar_supp_actors_movies.drop(columns=['wikidata_id'], inplace=True)
print(oscar_supp_actors_movies.shape)
print(oscar_supp_actors_movies.columns)

(83, 14)
Index(['Supporting Actor id', 'Wikipedia movie ID', 'Freebase movie ID',
       'Movie name', 'Movie release date', 'Movie box office revenue',
       'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres',
       'reviewScores', 'awardsReceived', 'awardsNominated', 'capitalCost'],
      dtype='object')


In [9]:
oscar_supp_actors_info = pd.merge(winning_actors_info, oscar_supp_actors_movies, left_on='page_id', right_on='Supporting Actor id')
print(oscar_supp_actors_info.columns)
oscar_supp_actors_info.drop(columns=['Freebase movie ID', 'sexLabel'], inplace=True)  #Remove the columns that are in character.csv
print(oscar_supp_actors_info.shape)

Index(['page_id', 'wikidata_id', 'sexLabel', 'nativeLanguageLabel',
       'countryOfCitizenshipLabel', 'ethnicGroupLabel', 'Actor name',
       'Supporting Actor id', 'Wikipedia movie ID', 'Freebase movie ID',
       'Movie name', 'Movie release date', 'Movie box office revenue',
       'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres',
       'reviewScores', 'awardsReceived', 'awardsNominated', 'capitalCost'],
      dtype='object')
(83, 19)


In [10]:
duplicates_with_different_ages = oscar_supp_actors_info.groupby('Actor name').filter(lambda x: x['page_id'].nunique() > 1)

if not duplicates_with_different_ages.empty:
    print("Duplicate names with different ages found:")
    print(duplicates_with_different_ages)
else:
    print("No duplicate names with different Actors id found.")

No duplicate names with different Actors id found.


In [11]:
#oscar_actors_movies = pd.concat([oscar_actors_movies, oscar_actors_extended])
winning_actors_info["Actors id"] = winning_actors_info["page_id"]
winning_actors_info = winning_actors_info.drop(columns=["page_id"])

character.columns


Index(['Wikipedia movie ID', 'Freebase movie ID', 'Movie release date',
       'Character name', 'Actor date of birth', ' Actor gender',
       'Actor height)', 'Actor ethnicity', 'Actor name',
       'Actor age at movie release', 'Freebase character',
       'Freebase character ID', 'Freebase actor ID'],
      dtype='object')

In [12]:
oscar_supp_actors_info = pd.merge(oscar_supp_actors_info, character[['Wikipedia movie ID', 'Actor name', 'Actor age at movie release', 'Actor date of birth', 'Actor ethnicity', 'Freebase actor ID']], on=['Wikipedia movie ID', 'Actor name'])


oscar_supp_actors_info.head(89)

,page_id,wikidata_id,nativeLanguageLabel,countryOfCitizenshipLabel,ethnicGroupLabel,Actor name,Supporting Actor id,Wikipedia movie ID,Movie name,Movie release date,...,Movie countries,Movie genres,reviewScores,awardsReceived,awardsNominated,capitalCost,Actor age at movie release,Actor date of birth,Actor ethnicity,Freebase actor ID
0,12561,Q150943,English,United States of America,NaN,Gene Hackman,12561,133574,Unforgiven,1992.0,...,United States of America,"Western, Drama",NaN,NaN,NaN,NaN,62.0,1930.0,White Americans,/m/039bp
1,15821,Q94123,English,United States of America,NaN,Jack Lemmon,15821,16040190,Mister Roberts,1955.0,...,United States of America,"Family Film, Comedy, War film",NaN,NaN,NaN,NaN,30.0,1925.0,NaN,/m/040z9
2,18617829,Q39792,American English,United States of America,NaN,Jack Nicholson,18617829,113427,Terms of Endearment,1983.0,...,United States of America,"Romantic comedy, Tragedy, Comedy film, Family ...",NaN,NaN,NaN,NaN,46.0,1937.0,Italian Americans,/m/09fb5
3,20913302,Q36949,English,United States of America,NaN,Robert De Niro,20913302,73875,The Godfather Part II,1974.0,...,United States of America,"Crime Fiction, Gangster Film, Drama, Crime Drama",NaN,NaN,NaN,NaN,31.0,1943.0,Italian Americans,/m/06cgy
4,16768,Q25144,English,United States of America,NaN,Kevin Spacey,16768,167857,The Usual Suspects,NaN,...,United States of America,"Thriller, Ensemble Film, Mystery, Crime Fictio...",NaN,NaN,NaN,NaN,NaN,1959.0,NaN,/m/048lv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,17173785,Q40572,English,Australia,NaN,Heath Ledger,17173785,4276475,The Dark Knight,2008.0,...,United States of America,"Crime Fiction, Thriller, Superhero movie, Crim...",NaN,NaN,NaN,NaN,29.0,1979.0,Irish Australians,/m/0237fw
67,8104201,Q76819,NaN,Germany,NaN,Christoph Waltz,8104201,633052,Inglourious Basterds,2009.0,...,United States of America,"Thriller, Black comedy, Action/Adventure, Dram...",NaN,NaN,NaN,NaN,52.0,1956.0,NaN,/m/026rm_y
68,8104201,Q76819,NaN,Germany,NaN,Christoph Waltz,8104201,31638720,Django Unchained,2012.0,...,United States of America,"Western, Spaghetti Western, Drama",NaN,NaN,NaN,NaN,56.0,1956.0,NaN,/m/026rm_y
69,331124,Q45772,British English,United Kingdom,NaN,Christian Bale,331124,23652051,The Fighter,2010.0,...,United States of America,"Sports, Biographical film, Biography, Drama, B...",NaN,NaN,NaN,NaN,36.0,1974.0,English people,/m/01wy5m


In [13]:
oscar_supporting_actors

,Supporting Actor id,film_id
0,477177,1138873
1,477177,1364205
2,477177,1364187
3,768336,44210
4,768351,62122
...,...,...
83,44849,56717294
84,9405817,62113200
85,68502180,66255918
86,266346,58342225


In [14]:
print('Shape:', oscar_supp_actors_info.shape)
print('Columns:', oscar_supp_actors_info.columns)
missing_values = oscar_supporting_actors['Supporting Actor id'][~oscar_supporting_actors['Supporting Actor id'].isin(oscar_supp_actors_info['Supporting Actor id'])]
print('Missing actresses in the cleaned dataset:', missing_values)
oscar_supp_actors_info.head(3)

Shape: (71, 23)
Columns: Index(['page_id', 'wikidata_id', 'nativeLanguageLabel',
       'countryOfCitizenshipLabel', 'ethnicGroupLabel', 'Actor name',
       'Supporting Actor id', 'Wikipedia movie ID', 'Movie name',
       'Movie release date', 'Movie box office revenue', 'Movie runtime',
       'Movie languages', 'Movie countries', 'Movie genres', 'reviewScores',
       'awardsReceived', 'awardsNominated', 'capitalCost',
       'Actor age at movie release', 'Actor date of birth', 'Actor ethnicity',
       'Freebase actor ID'],
      dtype='object')
Missing actresses in the cleaned dataset: 4       768351
9       779236
27      779475
36      438651
46      598484
61      158106
77      483360
78     1086198
79     2227406
80     2214063
81     2214063
82     1360740
83       44849
84     9405817
85    68502180
86      266346
87      171045
Name: Supporting Actor id, dtype: int64


,page_id,wikidata_id,nativeLanguageLabel,countryOfCitizenshipLabel,ethnicGroupLabel,Actor name,Supporting Actor id,Wikipedia movie ID,Movie name,Movie release date,...,Movie countries,Movie genres,reviewScores,awardsReceived,awardsNominated,capitalCost,Actor age at movie release,Actor date of birth,Actor ethnicity,Freebase actor ID
0,12561,Q150943,English,United States of America,NaN,Gene Hackman,12561,133574,Unforgiven,1992.0,...,United States of America,"Western, Drama",NaN,NaN,NaN,NaN,62.0,1930.0,White Americans,/m/039bp
1,15821,Q94123,English,United States of America,NaN,Jack Lemmon,15821,16040190,Mister Roberts,1955.0,...,United States of America,"Family Film, Comedy, War film",NaN,NaN,NaN,NaN,30.0,1925.0,NaN,/m/040z9
2,18617829,Q39792,American English,United States of America,NaN,Jack Nicholson,18617829,113427,Terms of Endearment,1983.0,...,United States of America,"Romantic comedy, Tragedy, Comedy film, Family ...",NaN,NaN,NaN,NaN,46.0,1937.0,Italian Americans,/m/09fb5


In [15]:
import plotly.express as px

# Prepare data for box office revenue evolution
oscar_revenue_data_corrected = oscar_supp_actors_info[['Movie box office revenue', 'Movie release date']].dropna()
oscar_revenue_data_corrected = oscar_revenue_data_corrected.dropna(subset=['Movie release date'])

# Group by year and calculate total box office revenue
oscar_revenue_data_corrected['Year'] = oscar_revenue_data_corrected['Movie release date']
revenue_by_year_corrected = oscar_revenue_data_corrected.groupby('Year')['Movie box office revenue'].sum().reset_index()

# Create a line chart
fig = px.line(
    revenue_by_year_corrected,
    x='Year',
    y='Movie box office revenue',
    title='Evolution of Movie Box Office Revenue Over the Years',
    labels={'Year': 'Year', 'Movie box office revenue': 'Box Office Revenue ($)'},
    line_shape='linear',
    markers=True
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(title='Year'),
    yaxis=dict(title='Total Box Office Revenue ($)', tickformat='$,.0f'),
    width=800, 
    height=600  
)

fig.show()

In [16]:
oscar_supp_actors_movies.columns

Index(['Supporting Actor id', 'Wikipedia movie ID', 'Freebase movie ID',
       'Movie name', 'Movie release date', 'Movie box office revenue',
       'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres',
       'reviewScores', 'awardsReceived', 'awardsNominated', 'capitalCost'],
      dtype='object')

In [17]:
import plotly.express as px
fig = px.histogram(oscar_supp_actors_info, 
                   x=oscar_supp_actors_info["Actor age at movie release"].value_counts().index,
                   y=oscar_supp_actors_info["Actor age at movie release"].value_counts().values, 
                   title="Number of Oscar winning actors by age at movie release",
                   labels={'x': 'Age at Movie Release', 'y': 'actors'},
                   nbins=17)
fig.update_layout( bargap=0.1)
fig.show()

In [18]:
if 'Actor age at movie release' in oscar_supp_actors_info.columns:
    # Count occurrences of each age
    age_distribution = oscar_supp_actors_info['Actor age at movie release'].value_counts().reset_index()
    age_distribution.columns = ['Actor age at movie release', 'count']
    
    # Sort by age for better visualization
    age_distribution = age_distribution.sort_values(by='Actor age at movie release')

    # Create the bar plot
    fig = px.bar(age_distribution, x='Actor age at movie release', y='count', title='Age Distribution', labels={'age': 'Age', 'count': 'Count'})
    fig.show()
else:
    print("The column 'age' does not exist in the provided file.")

In [19]:
# Print all unique values in the 'Actor ethnicity' column
unique_ethnicities = oscar_supp_actors_info['Actor ethnicity'].unique()

# Display the unique values
print(unique_ethnicities)

['White Americans' nan 'Italian Americans' 'African Americans'
 'Irish people' 'Serbian Americans' 'Mexican Americans' 'Jewish people'
 'English people' 'German Americans' 'Cambodian Americans'
 'Scottish people' 'Ukrainian Americans' 'American Jews' 'Puerto Ricans'
 'Irish Americans' 'Spaniards' 'Irish Australians']


In [20]:
fig = px.scatter_geo()
country_counts = oscar_supp_actors_info['countryOfCitizenshipLabel'].value_counts()
country_counts = country_counts.reset_index()
country_counts.columns = ['Country', 'Count']

fig = px.choropleth(country_counts, 
                    locations="Country", 
                    locationmode='country names', 
                    color="Count", 
                    hover_name="Country", 
                    title="Heatmap of Oscar Winning Actors by Country")
fig.update_geos(showcountries=True)
fig.show()

In [21]:
country_counts = oscar_supp_actors_info['countryOfCitizenshipLabel'].value_counts()
country_counts = country_counts.reset_index()
country_counts.columns = ['Country', 'Count']

fig = px.choropleth(country_counts, 
                    locations="Country", 
                    locationmode='country names', 
                    color="Count", 
                    hover_name="Country", 
                    title="Heatmap of Oscar Winning Actors by Country")
fig.update_geos(showcountries=True, visible=True)
fig.update_geos(lonaxis_range=[-10, 40], lataxis_range=[35, 70])
fig.show()

In [22]:
print('Average Capital Cost of the movies starring the actresses:', oscar_supp_actors_movies['capitalCost'].median()) 
print('Average Box Office revenue of the movies starring the actresses:', oscar_supp_actors_movies['Movie box office revenue'].median())
print('Average Benefits of the movies starring the actresses:', (oscar_supp_actors_movies['Movie box office revenue'] - oscar_supp_actors_movies['capitalCost']).median())

Average Capital Cost of the movies starring the actresses: 24000000.0
Average Box Office revenue of the movies starring the actresses: 54766923.0
Average Benefits of the movies starring the actresses: 136125887.5


In [23]:
oscar_supp_actors_movies_all = character[character['Freebase actor ID'].isin(oscar_supp_actors_info['Freebase actor ID'])]
oscar_supp_actors_movies_all = oscar_supp_actors_movies_all.drop(columns=['Freebase movie ID', 'Movie release date'])
oscar_supp_actors_movies_all = oscar_supp_actors_movies_all.merge(movie_cmu, on='Wikipedia movie ID', how='left')
print('Shape:', oscar_supp_actors_movies_all.shape)
print('Columns:', oscar_supp_actors_movies_all.columns)
print('Date boundary:', oscar_supp_actors_movies_all['Movie release date'].min(), '-', oscar_supp_actors_movies_all['Movie release date'].max())

Shape: (3110, 19)
Columns: Index(['Wikipedia movie ID', 'Character name', 'Actor date of birth',
       ' Actor gender', 'Actor height)', 'Actor ethnicity', 'Actor name',
       'Actor age at movie release', 'Freebase character',
       'Freebase character ID', 'Freebase actor ID', 'Freebase movie ID',
       'Movie name', 'Movie release date', 'Movie box office revenue',
       'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres'],
      dtype='object')
Date boundary: 1911.0 - 2013.0


In [24]:
# Create a set of unique (Actor name, Wikipedia movie ID) combinations for rewards
best_actors_movies = set(
    zip(oscar_supp_actors_info['Actor name'], oscar_supp_actors_info['Wikipedia movie ID'])
)

# Add a new column indicating if the movie is a Best Actress Reward
oscar_supp_actors_movies_all['Best Actors Reward'] = oscar_supp_actors_movies_all.apply(
    lambda row: (row['Actor name'], row['Wikipedia movie ID']) in best_actors_movies,
    axis=1
)

oscar_supp_actors_movies_all[oscar_supp_actors_movies_all['Actor name'] == 'Christopher Plummer']

,Wikipedia movie ID,Character name,Actor date of birth,Actor gender,Actor height),Actor ethnicity,Actor name,Actor age at movie release,Freebase character,Freebase character ID,Freebase actor ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Best Actors Reward
4,5894429,Jack,1929.0,M,1.778,NaN,Christopher Plummer,77.0,/m/0jwjks,/m/0h1yq64,/m/01ycbq,/m/0fc8w8,Closing the Ring,2007.0,NaN,117.0,English Language,United States of America,"Romantic drama, Romance Film, Drama, War film",False
9,30575,Dr. Goines,1929.0,M,1.778,NaN,Christopher Plummer,66.0,/m/040r0y7,/m/0h1zkq4,/m/01ycbq,/m/07j6w,Twelve Monkeys,1995.0,168839459.0,131.0,English Language,United States of America,"Thriller, Science Fiction, Future noir, Doomsd...",False
122,12877538,Mr. Caruthers,1929.0,M,1.778,NaN,Christopher Plummer,69.0,/m/04dcfww,/m/0h1zkt2,/m/01ycbq,/m/02x983k,The Clown at Midnight,1999.0,NaN,91.0,English Language,Canada,"Thriller, Horror",False
147,9283532,Eddie Chapman,1929.0,M,1.778,NaN,Christopher Plummer,36.0,/m/03jq6rc,/m/0h1zkt9,/m/01ycbq,/m/02pgkdn,Triple Cross,NaN,NaN,140.0,English Language,France,"Crime Fiction, Crime Thriller",False
160,10582947,Sir Charles,1929.0,M,1.778,NaN,Christopher Plummer,NaN,/m/03jpg6k,/m/0h1zkw9,/m/01ycbq,/m/02qjg31,Young Catherine,NaN,NaN,NaN,English Language,United States of America,"Costume drama, Historical fiction, Biopic [fea...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,920296,William Fawcett Robinson,1929.0,M,1.778,NaN,Christopher Plummer,50.0,/m/02vb8kp,/m/0cgwd1s,/m/01ycbq,/m/03p_np,Somewhere in Time,1980.0,NaN,108.0,English Language,United States of America,"Science Fiction, Indie, Tragedy, Romantic fant...",False
3033,5314234,John Klaxon,1929.0,M,1.778,NaN,Christopher Plummer,65.0,/m/0jz630,/m/0h1yqkl,/m/01ycbq,/m/0df1tr,Harrison Bergeron,1995.0,NaN,99.0,English Language,Canada,"Science Fiction, Satire, Dystopia",False
3065,385742,Commodus,1929.0,M,1.778,NaN,Christopher Plummer,34.0,/m/02vcd69,/m/0h1yq6z,/m/01ycbq,/m/0224yq,The Fall of the Roman Empire,1964.0,NaN,188.0,English Language,United States of America,"Sword and Sandal, Historical fiction, Epic, Pe...",False
3088,1138675,Aristotle,1929.0,M,1.778,NaN,Christopher Plummer,74.0,/m/0jt6fq,/m/046tc4d,/m/01ycbq,/m/049mql,Alexander,2004.0,167298192.0,214.0,English Language,United States of America,"LGBT, Biography, Adventure, Costume drama, His...",False


In [25]:
import plotly.express as px

# Prepare the data for the bar chart
oscar_movies_all = oscar_supp_actors_movies_all.copy()
oscar_movies_all['Movie genres'] = oscar_movies_all['Movie genres'].fillna('Unknown')
oscar_movies_all['Best Actors Reward'] = oscar_movies_all['Best Actors Reward'].map({True: 'Oscar-Winning', False: 'Non-Oscar-Winning'})

# Split the genres into individual rows for analysis
genres_split = oscar_movies_all.assign(Movie_genres_split=oscar_movies_all['Movie genres'].str.split(', ')).explode('Movie_genres_split')

# Group and count movies by genre and whether they won an Oscar
genre_counts = genres_split.groupby(['Movie_genres_split', 'Best Actors Reward']).size().reset_index(name='Count')

# Limit to the top 15 genres with the highest total count
top_genres = genre_counts.groupby('Movie_genres_split')['Count'].sum().nlargest(15).index
filtered_genre_counts = genre_counts[genre_counts['Movie_genres_split'].isin(top_genres)]

# Create the improved bar chart
fig = px.bar(
    filtered_genre_counts.sort_values('Count', ascending=False),
    x='Movie_genres_split',
    y='Count',
    color='Best Actors Reward',
    title='Top 15 Movie Genres (Oscar-Winning vs Non-Oscar-Winning)',
    labels={'Movie_genres_split': 'Movie Genre', 'Count': 'Number of Movies'},
    barmode='group'
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(tickangle=45, title='Movie Genre'),
    yaxis=dict(title='Number of Movies'),
    width=1000,  # Increase figure width
    height=500   # Adjust height
)

# Uncomment this to visualize
fig.show()

In [26]:
# Prepare data for box office revenue comparison
box_office_data = oscar_supp_actors_movies_all[['Movie box office revenue', 'Best Actors Reward']].dropna()

# Create a box plot to show the distribution of box office revenue
fig = px.box(
    box_office_data,
    x='Best Actors Reward',
    y='Movie box office revenue',
    title='Box Office Revenue Distribution (Oscar-Winning vs Non-Oscar-Winning)',
    labels={'Best Actors Reward': 'Movie Type', 'Movie box office revenue': 'Box Office Revenue ($)'},
    color='Best Actors Reward'
)

# Update layout for better readability⁄
fig.update_layout(
    xaxis=dict(title='Best Actors Reward'),
    yaxis=dict(title='Box Office Revenue ($)', tickformat='$,.0f'),
    width=800,  # Adjust figure width
    height=600  # Adjust figure height
)

# Uncomment this to visualize
fig.show()

In [27]:
scatter_data = oscar_supp_actors_info[['Actor age at movie release', 'Movie box office revenue']].dropna()


fig = px.scatter(
    scatter_data,
    x='Actor age at movie release',
    y='Movie box office revenue',
    title='Actress Age vs. Movie Box Office Revenue (Oscar-Winning Movies)',
    labels={
        'Actor age at movie release': 'Age at Movie Release',
        'Movie box office revenue': 'Box Office Revenue ($)'
    },
    size='Movie box office revenue',
    hover_data=['Actor age at movie release'],
    color_discrete_sequence=['purple']
)


fig.update_layout(
    xaxis=dict(title='Age at Movie Release'),
    yaxis=dict(title='Box Office Revenue ($)', tickformat='$,.0f'),
    width=800,  
    height=600  
)

fig.show()

In [28]:
genres_data = oscar_supp_actors_info[['Movie genres']].dropna()
genres_data = genres_data.assign(Movie_genres_split=genres_data['Movie genres'].str.split(', ')).explode('Movie_genres_split')

genre_counts = genres_data['Movie_genres_split'].value_counts().reset_index()
genre_counts.columns = ['Genre', 'Count']


fig = px.bar(
    genre_counts.head(10), 
    x='Genre',
    y='Count',
    title='Top 10 Genres in Oscar-Winning Movies',
    labels={'Genre': 'Movie Genre', 'Count': 'Number of Movies'},
    text='Count',
    color='Genre'
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(title='Movie Genre', tickangle=45),
    yaxis=dict(title='Number of Movies'),
    width=800,
    height=600 
)


fig.show()

In [29]:
# Ensure the date column is in datetime format for comparison
oscar_supp_actors_movies_all['Movie release date'] = pd.to_datetime(oscar_supp_actors_movies_all['Movie release date'], errors='coerce')

# Filter to only valid rows with release dates
oscar_movies_filtered = oscar_supp_actors_movies_all.dropna(subset=['Movie release date'])


def count_movies_before_first_oscar(group):
    # Sort movies by release date
    group = group.sort_values(by='Movie release date')
    # Find the first Oscar-winning movie
    first_oscar_date = group.loc[group['Best Actors Reward'], 'Movie release date'].min()
    # Count movies before the first Oscar-winning movie
    if pd.notnull(first_oscar_date):
        return group[group['Movie release date'] < first_oscar_date].shape[0]
    else:
        return group.shape[0]  # If no Oscar win, count all movies


movies_before_first_oscar = oscar_movies_filtered.groupby('Actor name').apply(count_movies_before_first_oscar).reset_index()
movies_before_first_oscar.columns = ['Actor name', 'Movies Before First Oscar']


fig = px.bar(
    movies_before_first_oscar.sort_values('Movies Before First Oscar', ascending=False),
    x='Actor name',
    y='Movies Before First Oscar',
    title='Number of Movies Before First Oscar per Actress',
    labels={'Actor name': 'Actor', 'Movies Before First Oscar': 'Movies Before First Oscar'},
    text='Movies Before First Oscar',
    color='Movies Before First Oscar',
    color_continuous_scale='Blues'
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(title='Actor', tickangle=45),
    yaxis=dict(title='Number of Movies'),
    width=1000,  # Adjust figure width
    height=600  # Adjust figure height
)

# Uncomment this line to display the graph
fig.show()

C:\Users\thier\AppData\Local\Temp\ipykernel_11300\568994408.py:20: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [30]:
print(oscar_supp_actors_movies_all.columns)

Index(['Wikipedia movie ID', 'Character name', 'Actor date of birth',
       ' Actor gender', 'Actor height)', 'Actor ethnicity', 'Actor name',
       'Actor age at movie release', 'Freebase character',
       'Freebase character ID', 'Freebase actor ID', 'Freebase movie ID',
       'Movie name', 'Movie release date', 'Movie box office revenue',
       'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres',
       'Best Actors Reward'],
      dtype='object')


In [31]:
#oscar_actors_movies2 = movie.merge(character, on='Wikipedia movie ID')
#oscar_actors_movies2 = oscar_actors_movies2.merge(winning_actors_info, on='Actor name')
#oscar_actors_movies2 = oscar_actors_movies2[oscar_actors_movies2['Actors id'].isin(oscar_winning_actors['Actors id'])]

# Group by the actor's name
grouped_by_actor = oscar_supp_actors_movies_all.groupby('Actor name').apply(lambda x: x)
grouped_by_actor = grouped_by_actor.drop(columns=['Actor name'])

# Get all movies done by the actors who won an Oscar
all_movies_by_oscar_winning_actors = movie_cmu[movie_cmu['Wikipedia movie ID'].isin(grouped_by_actor['Wikipedia movie ID'])]



C:\Users\thier\AppData\Local\Temp\ipykernel_11300\3434920353.py:6: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [32]:
import networkx as nx
import matplotlib.colors as mcolors

# Create a graph
G = nx.Graph()

# Add nodes and edges
colors = plt.cm.tab20(np.linspace(0, 1, len(grouped_by_actor.index.levels[0])))

for i, (actor, group) in enumerate(grouped_by_actor.groupby(level=0)):
    G.add_node(actor, type='actor', color=mcolors.to_hex(colors[i]))
    for _, row in group.iterrows():
        movie = row['Movie name']
        if not G.has_node(movie):
            G.add_node(movie, type='movie', color='#d3d3d3')  # Light gray color for movies
        G.add_edge(actor, movie, color=mcolors.to_hex(colors[i]))

# Adjust the positions of the nodes to improve visibility
pos = nx.spring_layout(G, k=5, iterations=100)

# Separate positions for actors and movies
for node in G.nodes(data=True):
    if node[1]['type'] == 'actor':
        pos[node[0]] = [pos[node[0]][0] * 3, pos[node[0]][1] * 3]  # Spread out actors more
    else:
        pos[node[0]] = [pos[node[0]][0] / 3, pos[node[0]][1] / 3]  # Center movies

# Draw the graph with improved visibility
plt.figure(figsize=(12, 12))
nx.draw(G, pos, with_labels=True, labels={n: n for n, d in G.nodes(data=True) if d['type'] == 'actor'}, 
        node_color=[node[1]['color'] for node in G.nodes(data=True)], 
        edge_color=[edge[2]['color'] for edge in G.edges(data=True)], 
        node_size=[700 if d['type'] == 'actor' else 300 for n, d in G.nodes(data=True)],  # Larger size for actors
        font_size=10)
plt.title('Network of Actors and Movies')
plt.show()


KeyboardInterrupt: 

In [ ]:
import matplotlib.colors as mcolors
import plotly.graph_objects as go

# Create a graph
G = nx.Graph()

# Add nodes and edges
colors = plt.cm.tab20(np.linspace(0, 1, len(grouped_by_actor.index.levels[0])))

for i, (actor, group) in enumerate(grouped_by_actor.groupby(level=0)):
    G.add_node(actor, type='actor', color=mcolors.to_hex(colors[i]))
    for _, row in group.iterrows():
        movie = row['Movie name']
        if not G.has_node(movie):
            G.add_node(movie, type='movie', color='#d3d3d3')  # Light gray color for movies
        G.add_edge(actor, movie, color=mcolors.to_hex(colors[i]))

# Adjust the positions of the nodes to improve visibility
pos = nx.spring_layout(G, k=10, iterations=200)

# Separate positions for actors and movies
for node in G.nodes(data=True):
    if node[1]['type'] == 'actor':
        pos[node[0]] = [pos[node[0]][0] * 3, pos[node[0]][1] * 3]  # Spread out actors more
    else:
        pos[node[0]] = [pos[node[0]][0] / 3, pos[node[0]][1] / 3]  # Center movies

# Create a plotly figure
fig = go.Figure()

# Add nodes
for node in G.nodes(data=True):
    if node[1]['type'] == 'actor':
        fig.add_trace(go.Scatter(
            x=[pos[node[0]][0]], y=[pos[node[0]][1]],
            mode='markers+text',
            marker=dict(size=10, color=node[1]['color']),
            text=node[0],
            textposition='top center',
            name=node[0]
        ))
    else:
        fig.add_trace(go.Scatter(
            x=[pos[node[0]][0]], y=[pos[node[0]][1]],
            mode='markers',
            marker=dict(size=5, color=node[1]['color']),
            name=node[0]
        ))

# Add edges
for edge in G.edges(data=True):
    fig.add_trace(go.Scatter(
        x=[pos[edge[0]][0], pos[edge[1]][0], None],
        y=[pos[edge[0]][1], pos[edge[1]][1], None],
        mode='lines',
        line=dict(width=0.5, color=edge[2]['color']),
        hoverinfo='none'
    ))

# Update layout for better visibility
fig.update_layout(
    title='Network of Actors and Movies',
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    width=1200,  # Increase the width
    height=800   # Increase the height
)

fig.show()

In [55]:
import plotly.express as px

# Count the number of movies each actor has done
actor_movie_counts = grouped_by_actor.groupby(level=0).size().reset_index(name='Movie Count')

# Create a bar chart
fig = px.bar(actor_movie_counts, x='Actor name', y='Movie Count', title='Number of Movies Each Actor Has Done')
fig.update_layout(xaxis_title='Actor', yaxis_title='Number of Movies', xaxis={'categoryorder':'total descending'})
fig.show()

In [56]:
# First, let's get the genres for each actor
genre_by_actor = grouped_by_actor.groupby('Actor name')['Movie genres'].apply(lambda x: ','.join([str(g) for g in x if pd.notna(g)])).reset_index()

# Split the genres string and explode to get one row per genre
genre_by_actor['Movie genres'] = genre_by_actor['Movie genres'].str.split(',')
genre_by_actor = genre_by_actor.explode('Movie genres')

# Clean up genres by stripping whitespace and removing empty strings
genre_by_actor['Movie genres'] = genre_by_actor['Movie genres'].str.strip()
genre_by_actor = genre_by_actor[genre_by_actor['Movie genres'] != '']

# Define genre mappings to group similar genres
genre_mappings = {
    'Drama': ['Drama', 'Family Drama', 'Comedy-drama', 'Melodrama', 'Docudrama'],
    'Comedy': ['Comedy', 'Romantic comedy', 'Comedy-drama', 'Black comedy'],
    'Action/Adventure': ['Action', 'Adventure', 'Action/Adventure', 'Thriller', 'War film'],
    'Crime': ['Crime Fiction', 'Crime film', 'Detective fiction', 'Film noir'],
    'Historical': ['Period piece', 'Historical fiction', 'Biography', 'History'],
    'Romance': ['Romance Film', 'Romantic drama', 'Romantic comedy'],
    'Musical': ['Musical', 'Music film', 'Opera film', 'Concert film'],
    'Horror': ['Horror', 'Supernatural horror', 'Slasher', 'Psychological horror'],
    'Western': ['Western', 'Spaghetti Western', 'Contemporary Western'],
    'SciFi/Fantasy': ['Science Fiction', 'Fantasy', 'Superhero film', 'Space opera'],
    'Documentary': ['Documentary', 'Docudrama', 'Documentary drama'],
    'Animation': ['Animation', 'Anime', 'Stop motion', 'Computer animation'],
    'Other': [] # Will catch any genres not matched above
}

# Map genres to their groups
def map_genre(genre):
    for group, genres in genre_mappings.items():
        if any(g.lower() in genre.lower() for g in genres):
            return group
    return 'Other'

genre_by_actor['Movie genres'] = genre_by_actor['Movie genres'].apply(map_genre)

# Count genres for each actor
genre_counts = genre_by_actor.groupby(['Actor name', 'Movie genres']).size().reset_index(name='count')

# Filter to only include genres that appear frequently
min_appearances = 5
frequent_genres = genre_counts.groupby('Movie genres')['count'].sum().reset_index()
frequent_genres = frequent_genres[frequent_genres['count'] >= min_appearances]['Movie genres']
genre_counts = genre_counts[genre_counts['Movie genres'].isin(frequent_genres)]

# Create a grouped bar chart
fig = px.bar(genre_counts, 
             x='Actor name', 
             y='count',
             color='Movie genres',
             title='Major Film Genres by Oscar-Winning Actor',
             labels={'count': 'Number of Movies', 'Actor name': 'Actor'},
             height=800)

# Update layout for better readability
fig.update_layout(
    xaxis={'categoryorder':'total descending'},
    showlegend=True,
    legend_title='Genre',
    barmode='stack',
    xaxis_tickangle=-45
)

fig.show()